# Data Quality Management

This is an initial assessment of the data quality. It is not part of CRISP-DM but of DMBOK.

At the end of the notebook I have eliminated the non-compliant rows. That is something that the client would be responsible for.

## Pre-Preprocessing dataset

In [ ]:
# Load needed librsries

import pandas as pd 

### Mounting files directly from google drive 

I have worked in the entire project in the cload (Colab) Please introduce your own path if you want to work on the project.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#Doing this, you don't need to upload the document in this noetbook (previous steps) because it is already in the drive
rawdf = pd.read_csv('/content/drive/My Drive/VIU/TFM/1st Iteration/quotes.csv', error_bad_lines=False, header=None)

rawdf.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,26,29,31,36,39,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,"attributed-no-source, best, life, love, mistak...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,You've gotta dance like there's nobody watchin...,William W. Purkey,"dance, heaven, hurt, inspirational, life, love...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,You know you're in love when you can't fall as...,Dr. Seuss,"attributed-no-source, dreams, love, reality, s...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A friend is someone who knows all about you an...,Elbert Hubbard,"friend, friendship, knowledge, love",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Darkness cannot drive out darkness: only light...,"Martin Luther King Jr., A Testament of Hope: T...","darkness, drive-out, hate, inspirational, ligh...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Cleanning up the dataframe

In [ ]:
#Copy in case changes are needed
df = rawdf.copy(deep=True)

In [ ]:
#Drop columns
df = df.drop([3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41], axis=1)

In [ ]:
#Let's rename the columns:
df = df.rename(columns={0:'Quotes',1:'Author',2:'Label'})

In [ ]:
# Authors won't be considered in this project
df = df.drop(['Author'], axis=1)

In [ ]:
#Let's load some quotes to try
df.iloc[345][0]
df.iloc[345345][0]
df.iloc[498000][0]

"We are all so close. We are godfather to each others' kids. I was the best man at Jesus' wedding."

In [ ]:
df.shape

(499709, 2)

In [ ]:
df.head(3)

,Quotes,Label
0,"I'm selfish, impatient and a little insecure. ...","attributed-no-source, best, life, love, mistak..."
1,You've gotta dance like there's nobody watchin...,"dance, heaven, hurt, inspirational, life, love..."
2,You know you're in love when you can't fall as...,"attributed-no-source, dreams, love, reality, s..."


### Dataset metadata




The metadata used in the dataset is registered:

* Type of data per attribute
* Value range presented
* Value characteristics

In [ ]:
#Type of data

df.info()

print("-------------------------------------")
print(f"Amount of columns: {df.shape[0]}")
print(f"Amount of rows: {df.shape[1]}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499709 entries, 0 to 499708
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Quotes  499708 non-null  object
 1   Label   499646 non-null  object
dtypes: object(2)
memory usage: 7.6+ MB
-------------------------------------
Amount of columns: 499709
Amount of rows: 2


In [ ]:
#Add column length
df['Length Quote'] = df.Quotes.str.len()

In [ ]:
#Sort quotes to see longest ones
sortquotes = df.sort_values(['Length Quote'], ascending=False)
sortquotes.head(7)

,Quotes,Label,Length Quote
334476,"I imagined my coffin being closed, and the scr...",death-and-dying,3999.0
301132,"There is no explaining the ""pure"" experience. ...","enlightenment, liberalism, political-science",3995.0
214689,Take the famous slogan on the atheist bus in L...,"atheism, atheist-bus, god, hope, new-atheism, ...",3986.0
234843,"As he sat up, he heard soft dripping sounds fr...","blood, eyes-like-blue-fire, ghost, gothic, hor...",3979.0
291945,a serious contender for my book of year. I can...,crime,3976.0
104891,From the Bridge”The Importance of HistoryNot a...,"american, cuba, history, informative",3972.0
98507,"Having been a Ship’s Captain, a Naval Officer ...","art, books, captain-hank-bracker, cuban-histor...",3966.0


## Initial Quality Assessment

The client defined certain metrics such as:
* quote and label completeness
* quote cleaness
* label cleaness
* quotes and label length

### Completeness - Quotes & Label

In [ ]:
number_rows = df.shape[0]
number_columns = df.shape[1]

#Checking amount of nans per atribute

row_NaN1 = df.Quotes.isnull().sum().sum()
row_NaN2 = df.Label.isnull().sum().sum()

print(f"Amount of Quotes with NaN: {row_NaN1}")
print(f"Percentage of Quotes with NaN: {(row_NaN1/number_rows)*100}%")
print(f"Amount of Labels with NaN: {row_NaN2}")
print(f"Percentage of Labels with NaN: {(row_NaN2/number_rows)*100}%")

Amount of Quotes with NaN: 1
Percentage of Quotes with NaN: 0.00020011646778425043%
Amount of Labels with NaN: 63
Percentage of Labels with NaN: 0.012607337470407778%


In [ ]:
df[df['Label'].isnull()]==False

,Quotes,Label,Length Quote
35741,False,False,False
35978,False,False,False
100876,False,False,False
142392,False,False,False
168298,False,False,False
...,...,...,...
434569,False,False,False
447058,False,False,False
457045,False,False,False
468842,False,False,False


### FormatValid - Quotes

To check the format, it is important to convert column Quotes to strings:


In [ ]:
df['Quotes'] = df.Quotes.astype(str)

**Number of hashtags**

In [ ]:
# Calculate amount hashtags and rows with hashtags
list_hashtags = []
counthash = 0
countrow = 0
for ind in df.index:
  for word in df['Quotes'][ind].split():
    if (word[:1] == '#'):
      counthash = counthash + 1
      list_hashtags.append(word)
  for word in df['Quotes'][ind].split():
    if (word[:1] == '#'):
      countrow = countrow + 1
      break

#list_hashtags
#counthash
countrow

315

**Number of mentions**

In [ ]:
# Calculate amount mentions and rows with mentions
list_mentions = []
countment = 0
countrow = 0
for ind in df.index:
  for word in df['Quotes'][ind].split():
    if (word[:1] == '@'):
      countment = countment + 1
      list_mentions.append(word)
  for word in df['Quotes'][ind].split():
    if (word[:1] == '@'):
      countrow = countrow + 1
      break

#list_mentions
#countment
countrow

24

**Number of links**

In [ ]:
#Count number of links present in the dataset
countrow = 0
substring = 'http'
for ind in df.index:
  for word in df['Quotes'][ind].split():
    if word.count(substring):
      countrow = countrow + 1

countrow

0

**Checking the combination of hashtags / mentions**

In [ ]:
#Check the combination hashtags mentions links
for ind in df.index:
  for word in df['Quotes'][ind].split():
    if (word[:1] == '#') or (word[:1] == '@'):
      countrow = countrow + 1
      break

countrow

336

### FormatValid - Labels

To check the format, it is important to convert column Labels to strings:

In [ ]:
df['Label'] = df.Label.astype(str)

In [ ]:
df.shape

(499709, 3)

**Number of Hashtags**

In [ ]:
# Calculate amount hashtags and rows with hashtags
list_hashtags = []
counthash = 0
countrow = 0
for ind in df.index:
  for word in df['Label'][ind].split():
    if (word[:1] == '#'):
      counthash = counthash + 1
      list_hashtags.append(word)
  for word in df['Label'][ind].split():
    if (word[:1] == '#'):
      countrow = countrow + 1
      break

#list_hashtags
#counthash
countrow

5

**Number of Methods**

In [ ]:
# Calculate amount mentions and rows with mentions
list_mentions = []
countment = 0
countrow = 0
for ind in df.index:
  for word in df['Label'][ind].split():
    if (word[:1] == '@'):
      countment = countment + 1
      list_mentions.append(word)
  for word in df['Label'][ind].split():
    if (word[:1] == '@'):
      countrow = countrow + 1
      break

#list_mentions
#countment
countrow

0

**Number of Links**

In [ ]:
#Count number of links present in the dataset
countrow = 0
substring = 'http'
for ind in df.index:
  for word in df['Label'][ind].split():
    if word.count(substring):
      countrow = countrow + 1

countrow

5

**Checking the combination of hashtags / mentions**

In [ ]:
#Check the combination hashtags mentions links
for ind in df.index:
  for word in df['Label'][ind].split():
    if (word[:1] == '#') or (word[:1] == '@'):
      countrow = countrow + 1
      break

countrow

10

### AdjustedValues - Quotes

In [ ]:
#Number of quotes longer than requirement
len(df[df['Length Quote']>280])

93529

In [ ]:
#Percentage quotes
len(df[df['Length Quote']>280])/df.shape[0]*100

18.71669311539316

In [ ]:
#Number of quotes shorter than requirement
len(df[df['Length Quote']<21])

2190

In [ ]:
#Percentage quotes
len(df[df['Length Quote']<21])/df.shape[0]*100

0.43825506444750845

In [ ]:
df.head(3)

,Quotes,Label,Length Quote
0,"I'm selfish, impatient and a little insecure. ...","attributed-no-source, best, life, love, mistak...",202.0
1,You've gotta dance like there's nobody watchin...,"dance, heaven, hurt, inspirational, life, love...",149.0
2,You know you're in love when you can't fall as...,"attributed-no-source, dreams, love, reality, s...",102.0


### AdjustedValues - Label

In [ ]:
#create empty column 'Label Count'
df['Label Count'] = 3

In [ ]:
for ind, row in df.iterrows():
  df['Label Count'][ind] = len(row['Label'].split())

In [ ]:
#Sort quotes to see longest ones
sortquotes = df.sort_values(['Label Count'], ascending=False)
sortquotes.head(7)

,Quotes,Label,Length Quote,Label Count
50602,If the surprise outcome of the recent UK refer...,"antagonism, antagonistic, austerity, better-in...",502.0,303
20209,Belief is a wonderful way to pass the time unt...,"aetheism, agnostic, agnostic-atheism, agnostic...",67.0,302
83153,"Casting a curious gaze down on planet Earth, e...","artificial-intelligence, artificial-life, arti...",270.0,223
129644,"When enforcing our boundaries, first and forem...","accept-yourself, acceptance, amy-leigh-mercree...",338.0,220
50832,Pick a leader who will make their citizens pro...,"administration, airlines, bail, bail-out, bank...",481.0,210
25262,Positive thinking is powerful thinking. If you...,"achieve, achievement, achievement-and-attitude...",221.0,196
15126,How initially 'to get her in the sack' and sub...,"advice-for-daily-living, advice-for-life, advi...",199.0,179


In [ ]:
#Number of registers with mnore than one Label
len(df[df['Label Count']==1])

72492

In [ ]:
#Percentage registers with mnore than one Label
len(df[df['Label Count']>10])/df.shape[0]*100

8.967018804944479

## Data elimination out of IQA

Now we eliminate all non-compliant rows.

In [ ]:
df.shape

(499709, 4)

In [ ]:
df = df[df.Label != 'nan']
df = df[df.Quotes != 'nan']

In [ ]:
df.shape

(499646, 4)

Drop regarding identifier FormatValid

In [ ]:
#Check the combination hashtags mentions links
count = 0
for ind in df.index:
  for word in df['Quotes'][ind].split():
    if (word[:1] == '#') or (word[:1] == '@'):
      df.drop(ind, inplace=True)
      count = count + 1
      break

count

336

In [ ]:
#Check the combination hashtags mentions links
count = 0
for ind in df.index:
  for word in df['Label'][ind].split():
    if (word[:1] == '#') or (word[:1] == '@'):
      df.drop(ind, inplace=True)
      count = count + 1
      break

count

5

In [ ]:
#Count number of links present in the dataset
count = 0
substring = 'http'
for ind in df.index:
  for word in df['Label'][ind].split():
    if word.count(substring):
      df.drop(ind, inplace=True)
      count = count + 1

count

5

In [ ]:
df.shape

(499300, 4)

Drop regarding identifier AdjustedValues

In [ ]:
#Drop rows with more than 280 characters
df = df.drop(df[df['Length Quote']>280].index)

In [ ]:
#Drop rows with less than 21 characters
df = df.drop(df[df['Length Quote']<21].index)

In [ ]:
#Column length quote dropped
df = df.drop(['Length Quote'], axis=1)

In [ ]:
df.shape

(403658, 3)

In [ ]:
df.head(3)

,Quotes,Label,Label Count
0,"I'm selfish, impatient and a little insecure. ...","attributed-no-source, best, life, love, mistak...",8
1,You've gotta dance like there's nobody watchin...,"dance, heaven, hurt, inspirational, life, love...",7
2,You know you're in love when you can't fall as...,"attributed-no-source, dreams, love, reality, s...",5


In [ ]:
#Drop rows with more than 10 labels
df = df.drop(df[df['Label Count']>10].index)

In [ ]:
#Column Label Count  dropped
df = df.drop(['Label Count'], axis=1)

In [ ]:
df.shape

(367185, 2)

Download file

In [ ]:
from google.colab import files
df.to_csv('quotes_phase0.csv', index=False)
files.download("quotes_phase0.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>